In [76]:
import numpy as np
import scipy.io
train_mat = scipy.io.loadmat('train_data.mat') 
test_mat = scipy.io.loadmat('test_data.mat') 

In [77]:
def get_data_from_mat(train_file,test_file):
    train_mat = scipy.io.loadmat(train_file) 
    test_mat = scipy.io.loadmat(test_file) 
    print(train_mat.keys())
    print(test_mat.keys())
    train_np = np.array(train_mat['train_data']).transpose(2,0,1)
    test_np = np.array(test_mat['test_data']).transpose(2,0,1)
    print(train_np.shape)
    print(test_np.shape)
    return  train_np, test_np


train_data, test_data = get_data_from_mat('train_data.mat','test_data.mat')

dict_keys(['__header__', '__version__', '__globals__', 'train_data'])
dict_keys(['__header__', '__version__', '__globals__', 'test_data'])
(112, 90, 90)
(28, 90, 90)


In [78]:
print(train_data[0])

[[0.         0.         0.13152513 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.13152513 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.28060623]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.28060623 0.         0.        ]]


In [79]:
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact

def visualize(image):  
    #plt.figure("sample", (12, 6))
    #plt.subplot(1, 2, 1)
    plt.imshow(image, cmap="gray")    
    #plt.subplot(1, 2, 2)
    #plt.imshow(image, cmap="gray")
    plt.show()      


#@interact
def show_ds(ds):
    print("ds shape:",ds.shape)
    print("ds max:",np.max(ds))
    print("ds min:",np.min(ds))
    print("ds average:",np.average(ds))
    @interact
    def visualize_set(scan_index=(0,len(ds)-1)):
        #print(scan_index)
        visualize(ds[scan_index,:,:])

show_ds(train_data)

ds shape: (112, 90, 90)
ds max: 1.0
ds min: 0.0
ds average: 0.024116160549305543


interactive(children=(IntSlider(value=55, description='scan_index', max=111), Output()), _dom_classes=('widget…

In [80]:
show_ds(test_data)

ds shape: (28, 90, 90)
ds max: 1.0
ds min: 0.0
ds average: 0.024994026906171023


interactive(children=(IntSlider(value=13, description='scan_index', max=27), Output()), _dom_classes=('widget-…

In [81]:
import pandas as pd
import numpy as np
import scipy.io
mat = scipy.io.loadmat('train_data.mat')
pqr=pd.Series(mat)
pd.DataFrame({'label':pqr.index, 'list':pqr.values})

,label,list
0,__header__,"b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Crea..."
1,__version__,1.0
2,__globals__,[]
3,train_data,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."


In [82]:
print(pqr)

__header__     b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Crea...
__version__                                                  1.0
__globals__                                                   []
train_data     [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...
dtype: object


In [83]:
data = np.genfromtxt("train.csv", delimiter=",")
data = data[1:]
X = np.delete(data, 1, 1)
X = np.delete(X, 0, 1)

Y = np.delete(data, 0, 1)
Y = np.delete(Y, 2, 1)
Y = np.delete(Y, 1, 1)

In [84]:
print(f"Shape X: {X.shape}")
print(f"Shape Y: {Y.shape}")

Shape X: (112, 2)
Shape Y: (112, 1)


In [85]:
import random
import os

os.environ['PYTHONHASHSEED']=str(2)

import tensorflow as tf
import tensorflow.keras.layers as L

In [86]:
def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(2)
   tf.random.set_seed(2)
   np.random.seed(2)
   random.seed(2)

reset_random_seeds()

In [87]:
train_data.shape

(112, 90, 90)

In [88]:
train_data = train_data.reshape(train_data.shape[0],90,90,1)

In [89]:
train_data.shape

(112, 90, 90, 1)

In [90]:
reset_random_seeds()
model = tf.keras.Sequential([
    L.InputLayer(input_shape=(90,90,1)),
    L.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    L.BatchNormalization(),
    L.MaxPooling2D((2, 2)),
    L.Conv2D(64, (3, 3), activation='relu'),
    L.MaxPooling2D((2, 2)),
    L.Conv2D(128, (3, 3), activation='relu'),
    L.MaxPooling2D((2, 2)),
    L.Flatten(),
    L.Dense(64, activation='relu'),
    L.Dropout(rate=0.5),
    L.Dense(1, activation='relu')
])

sgd = tf.keras.optimizers.SGD(momentum=0.9)

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])


## Stop training when validation loss reach 110
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_loss')<110):
            print("\nReached 110 val_loss so cancelling training!")
            self.model.stop_training = True
        
callback = myCallback()


model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 88, 88, 32)        320       
_________________________________________________________________
batch_normalization_3 (Batch (None, 88, 88, 32)        128       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 44, 44, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 42, 42, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 21, 21, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 19, 19, 128)       73856     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 9, 9, 128)        

In [91]:
history = model.fit(
    train_data, Y, epochs=500, validation_split=0.1, batch_size=64, callbacks=[callback]
)

Epoch 1/500
2/2 [==============================] - 0s 176ms/step - loss: 1565.5743 - mae: 33.7804 - val_loss: 4162.2544 - val_mae: 63.9758
Epoch 2/500
2/2 [==============================] - 0s 46ms/step - loss: 910.9269 - mae: 24.5266 - val_loss: 4323.3647 - val_mae: 65.2231
Epoch 3/500
2/2 [==============================] - 0s 45ms/step - loss: 558.0158 - mae: 20.8642 - val_loss: 4394.9644 - val_mae: 65.7698
Epoch 4/500
2/2 [==============================] - 0s 47ms/step - loss: 683.1244 - mae: 20.8372 - val_loss: 4362.4731 - val_mae: 65.5223
Epoch 5/500
2/2 [==============================] - 0s 50ms/step - loss: 546.5639 - mae: 20.3331 - val_loss: 4279.2495 - val_mae: 64.8840
Epoch 6/500
2/2 [==============================] - 0s 50ms/step - loss: 530.3432 - mae: 19.6990 - val_loss: 4254.9497 - val_mae: 64.6965
Epoch 7/500
2/2 [==============================] - 0s 50ms/step - loss: 589.5125 - mae: 20.6452 - val_loss: 4314.7446 - val_mae: 65.1571
Epoch 8/500
2/2 [======================

2/2 [==============================] - 0s 49ms/step - loss: 180.8288 - mae: 10.6719 - val_loss: 3878.3728 - val_mae: 61.7203
Epoch 61/500
2/2 [==============================] - 0s 43ms/step - loss: 175.2529 - mae: 10.1239 - val_loss: 3818.7141 - val_mae: 61.2356
Epoch 62/500
2/2 [==============================] - 0s 48ms/step - loss: 156.8721 - mae: 9.7612 - val_loss: 3824.6101 - val_mae: 61.2840
Epoch 63/500
2/2 [==============================] - 0s 49ms/step - loss: 148.1518 - mae: 9.8513 - val_loss: 3851.2109 - val_mae: 61.5009
Epoch 64/500
2/2 [==============================] - 0s 44ms/step - loss: 141.5205 - mae: 9.2078 - val_loss: 3838.4258 - val_mae: 61.3973
Epoch 65/500
2/2 [==============================] - 0s 49ms/step - loss: 151.6012 - mae: 9.5439 - val_loss: 3812.2668 - val_mae: 61.1845
Epoch 66/500
2/2 [==============================] - 0s 53ms/step - loss: 162.8473 - mae: 9.8775 - val_loss: 3799.2539 - val_mae: 61.0786
Epoch 67/500
2/2 [==============================] - 

Epoch 120/500
2/2 [==============================] - 0s 55ms/step - loss: 112.9201 - mae: 8.3821 - val_loss: 3614.5901 - val_mae: 59.5623
Epoch 121/500
2/2 [==============================] - 0s 56ms/step - loss: 138.7833 - mae: 9.1353 - val_loss: 3622.6846 - val_mae: 59.6303
Epoch 122/500
2/2 [==============================] - 0s 44ms/step - loss: 138.8939 - mae: 8.8944 - val_loss: 3401.5342 - val_mae: 57.7478
Epoch 123/500
2/2 [==============================] - 0s 44ms/step - loss: 144.0695 - mae: 9.7443 - val_loss: 3414.5403 - val_mae: 57.8600
Epoch 124/500
2/2 [==============================] - 0s 47ms/step - loss: 92.6952 - mae: 8.0596 - val_loss: 3591.7522 - val_mae: 59.3718
Epoch 125/500
2/2 [==============================] - 0s 47ms/step - loss: 146.0538 - mae: 9.2645 - val_loss: 3599.2539 - val_mae: 59.4359
Epoch 126/500
2/2 [==============================] - 0s 44ms/step - loss: 161.9571 - mae: 9.6525 - val_loss: 3344.7000 - val_mae: 57.2566
Epoch 127/500
2/2 [================

2/2 [==============================] - 0s 60ms/step - loss: 127.0713 - mae: 8.7323 - val_loss: 2784.6660 - val_mae: 52.1439
Epoch 180/500
2/2 [==============================] - 0s 53ms/step - loss: 92.9425 - mae: 6.9395 - val_loss: 2816.6162 - val_mae: 52.4480
Epoch 181/500
2/2 [==============================] - 0s 57ms/step - loss: 93.3382 - mae: 7.3633 - val_loss: 2846.0417 - val_mae: 52.7278
Epoch 182/500
2/2 [==============================] - 0s 57ms/step - loss: 83.4409 - mae: 6.7990 - val_loss: 2707.6035 - val_mae: 51.3991
Epoch 183/500
2/2 [==============================] - 0s 59ms/step - loss: 77.1223 - mae: 6.7210 - val_loss: 2756.7415 - val_mae: 51.8768
Epoch 184/500
2/2 [==============================] - 0s 54ms/step - loss: 130.7112 - mae: 8.5250 - val_loss: 2841.5959 - val_mae: 52.6917
Epoch 185/500
2/2 [==============================] - 0s 52ms/step - loss: 78.2770 - mae: 6.5687 - val_loss: 2766.3262 - val_mae: 51.9746
Epoch 186/500
2/2 [==============================] - 

Epoch 239/500
2/2 [==============================] - 0s 42ms/step - loss: 97.9303 - mae: 7.5965 - val_loss: 1874.9801 - val_mae: 42.4774
Epoch 240/500
2/2 [==============================] - 0s 43ms/step - loss: 88.0333 - mae: 6.9907 - val_loss: 1820.1813 - val_mae: 41.8267
Epoch 241/500
2/2 [==============================] - 0s 50ms/step - loss: 93.1370 - mae: 6.9945 - val_loss: 2026.0482 - val_mae: 44.2256
Epoch 242/500
2/2 [==============================] - 0s 49ms/step - loss: 110.6626 - mae: 7.2935 - val_loss: 1929.0249 - val_mae: 43.1169
Epoch 243/500
2/2 [==============================] - 0s 51ms/step - loss: 62.7149 - mae: 6.1615 - val_loss: 1631.6549 - val_mae: 39.5217
Epoch 244/500
2/2 [==============================] - 0s 44ms/step - loss: 130.9126 - mae: 8.7655 - val_loss: 1857.3666 - val_mae: 42.2869
Epoch 245/500
2/2 [==============================] - 0s 44ms/step - loss: 116.8148 - mae: 7.6926 - val_loss: 1982.1279 - val_mae: 43.7405
Epoch 246/500
2/2 [===================

Epoch 299/500
2/2 [==============================] - 0s 42ms/step - loss: 80.2424 - mae: 6.4945 - val_loss: 770.5341 - val_mae: 26.2718
Epoch 300/500
2/2 [==============================] - 0s 46ms/step - loss: 94.3352 - mae: 7.4182 - val_loss: 918.2161 - val_mae: 28.9638
Epoch 301/500
2/2 [==============================] - 0s 43ms/step - loss: 75.6318 - mae: 6.4439 - val_loss: 1076.6376 - val_mae: 31.5871
Epoch 302/500
2/2 [==============================] - 0s 44ms/step - loss: 97.7309 - mae: 6.9113 - val_loss: 937.4380 - val_mae: 29.2697
Epoch 303/500
2/2 [==============================] - 0s 44ms/step - loss: 75.4175 - mae: 6.3888 - val_loss: 935.0950 - val_mae: 29.2215
Epoch 304/500
2/2 [==============================] - 0s 48ms/step - loss: 74.4641 - mae: 6.6277 - val_loss: 971.1479 - val_mae: 29.8385
Epoch 305/500
2/2 [==============================] - 0s 61ms/step - loss: 96.6483 - mae: 7.3654 - val_loss: 868.3317 - val_mae: 28.0551
Epoch 306/500
2/2 [============================

2/2 [==============================] - 0s 53ms/step - loss: 69.6517 - mae: 6.1945 - val_loss: 268.7124 - val_mae: 14.4740
Epoch 360/500
2/2 [==============================] - 0s 56ms/step - loss: 79.8276 - mae: 6.6463 - val_loss: 287.1634 - val_mae: 15.0484
Epoch 361/500
2/2 [==============================] - 0s 57ms/step - loss: 72.5132 - mae: 6.1256 - val_loss: 379.2899 - val_mae: 17.5600
Epoch 362/500
2/2 [==============================] - 0s 56ms/step - loss: 77.8938 - mae: 6.4137 - val_loss: 421.2457 - val_mae: 18.5678
Epoch 363/500
2/2 [==============================] - 0s 57ms/step - loss: 84.5184 - mae: 6.9004 - val_loss: 278.6893 - val_mae: 14.7441
Epoch 364/500
2/2 [==============================] - 0s 56ms/step - loss: 111.6572 - mae: 7.7136 - val_loss: 276.3309 - val_mae: 14.6569
Epoch 365/500
2/2 [==============================] - 0s 56ms/step - loss: 84.2046 - mae: 6.7863 - val_loss: 401.0660 - val_mae: 18.0885
Epoch 366/500
2/2 [==============================] - 0s 57ms/

2/2 [==============================] - 0s 49ms/step - loss: 83.6256 - mae: 6.8300 - val_loss: 191.5955 - val_mae: 11.1086
Epoch 420/500
2/2 [==============================] - 0s 52ms/step - loss: 85.0022 - mae: 6.5388 - val_loss: 159.0439 - val_mae: 9.9606
Epoch 421/500
2/2 [==============================] - 0s 43ms/step - loss: 90.9139 - mae: 7.0719 - val_loss: 183.2424 - val_mae: 10.7946
Epoch 422/500
2/2 [==============================] - 0s 42ms/step - loss: 92.1742 - mae: 7.0257 - val_loss: 281.0243 - val_mae: 14.6704
Epoch 423/500
2/2 [==============================] - 0s 43ms/step - loss: 99.3484 - mae: 7.5848 - val_loss: 239.5832 - val_mae: 13.2136
Epoch 424/500
2/2 [==============================] - 0s 49ms/step - loss: 112.9667 - mae: 6.9656 - val_loss: 158.8138 - val_mae: 9.9687
Epoch 425/500
2/2 [==============================] - 0s 57ms/step - loss: 78.8411 - mae: 6.6873 - val_loss: 154.2377 - val_mae: 9.7941
Epoch 426/500
2/2 [==============================] - 0s 55ms/ste

2/2 [==============================] - 0s 57ms/step - loss: 45.8736 - mae: 5.0153 - val_loss: 238.7526 - val_mae: 12.9528
Epoch 480/500
2/2 [==============================] - 0s 55ms/step - loss: 91.5437 - mae: 7.4809 - val_loss: 192.6549 - val_mae: 11.3459
Epoch 481/500
2/2 [==============================] - 0s 56ms/step - loss: 67.4306 - mae: 6.3327 - val_loss: 135.0803 - val_mae: 8.8868
Epoch 482/500
2/2 [==============================] - 0s 43ms/step - loss: 115.4985 - mae: 7.9267 - val_loss: 188.2481 - val_mae: 11.1421
Epoch 483/500
2/2 [==============================] - 0s 43ms/step - loss: 83.7398 - mae: 6.5713 - val_loss: 250.3968 - val_mae: 13.3055
Epoch 484/500
2/2 [==============================] - 0s 46ms/step - loss: 83.8834 - mae: 6.6834 - val_loss: 198.7735 - val_mae: 11.3937
Epoch 485/500
2/2 [==============================] - 0s 54ms/step - loss: 67.3887 - mae: 5.8179 - val_loss: 168.4280 - val_mae: 10.4261
Epoch 486/500
2/2 [==============================] - 0s 54ms/s

In [92]:
test_data = test_data.reshape(test_data.shape[0],90,90,1)


In [93]:
pred=model.predict(test_data)
print(pred)

[[24.668821]
 [19.086489]
 [18.351332]
 [20.43825 ]
 [22.717564]
 [23.613379]
 [44.321823]
 [39.27758 ]
 [45.609646]
 [59.143345]
 [65.64745 ]
 [66.76102 ]
 [72.38603 ]
 [66.36529 ]
 [15.914827]
 [18.5743  ]
 [16.242485]
 [33.26451 ]
 [22.74307 ]
 [27.73098 ]
 [46.97158 ]
 [58.94708 ]
 [50.658524]
 [67.75148 ]
 [48.500248]
 [53.125446]
 [54.314137]
 [57.50042 ]]


In [94]:
import csv

header = ['id', 'age']

with open('example_submission.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    writer.writerow(header)
    
    i=0
    for p in pred:
        writer.writerow([i+1, pred[i][0]])
        i = i+1
        
    
    

PermissionError: [Errno 13] Permission denied: 'example_submission.csv'